# Part II

In [1]:
!createdb -U student financial_db  # Create a new financial_db database


createdb: error: database creation failed: ERROR:  database "financial_db" already exists


**Drop the financial_db database if it exists**

In [2]:
!psql -U student -l


                               List of databases
     Name     |  Owner   | Encoding | Collate |  Ctype  |   Access privileges   
--------------+----------+----------+---------+---------+-----------------------
 financial_db | student  | UTF8     | C.UTF-8 | C.UTF-8 | 
 postgres     | postgres | UTF8     | C.UTF-8 | C.UTF-8 | 
 template0    | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
              |          |          |         |         | postgres=CTc/postgres
 template1    | postgres | UTF8     | C.UTF-8 | C.UTF-8 | =c/postgres          +
              |          |          |         |         | postgres=CTc/postgres
 week6        | student  | UTF8     | C.UTF-8 | C.UTF-8 | 
(5 rows)



**List all the database**

In [3]:
%load_ext sql

**Load the SQL extension to enable SQL code execution in Jupyter Notebook**

In [4]:
%sql postgresql://student@localhost/financial_db  

**Connect to the financial_db database using the student user**

In [5]:
%%sql
DROP TABLE IF EXISTS SUB CASCADE;

CREATE TABLE SUB (
    adsh VARCHAR(20) NOT NULL PRIMARY KEY,  -- Primary key, uniquely identifies each submission
    cik NUMERIC(10) NOT NULL,  -- Company identifier
    name VARCHAR(150) NOT NULL,  -- Company name
    sic NUMERIC(4),
    countryba VARCHAR(2),  -- Country of business address
    stprba VARCHAR(2),
    cityba VARCHAR(30),
    zipba VARCHAR(10),
    bas1 VARCHAR(40),
    bas2 VARCHAR(40),
    baph VARCHAR(20),
    countryma VARCHAR(2),
    stprma VARCHAR(2),
    cityma VARCHAR(30),
    zipma VARCHAR(10),
    mas1 VARCHAR(40),
    mas2 VARCHAR(40),
    countryinc VARCHAR(2),
    stprinc VARCHAR(2),
    ein NUMERIC(10),
    former VARCHAR(150),
    changed VARCHAR(8),
    afs VARCHAR(5),
    wksi BOOLEAN NOT NULL,
    fye VARCHAR(4),
    form VARCHAR(20) NOT NULL,
    period DATE,
    fy NUMERIC(4),
    fp VARCHAR(2),
    filed DATE NOT NULL,
    accepted DATE NOT NULL,
    prevrpt BOOLEAN NOT NULL,
    detail BOOLEAN NOT NULL,
    instance VARCHAR(32) NOT NULL,
    nciks NUMERIC(4) NOT NULL,
    aciks VARCHAR(120),
    pubfloatusd NUMERIC(15,2),
    floatdate DATE,
    floataxis VARCHAR(255),
    floatmems NUMERIC(1)
);


 * postgresql://student@localhost/financial_db
Done.
Done.


[]

**Drop the existing table if it exists, then create a new table 'SUB' with various columns for storing company submission details, including identifiers, addresses, and financial data**

In [6]:
%%sql
DROP TABLE IF EXISTS TAG CASCADE;

CREATE TABLE TAG (
    tag VARCHAR(256) NOT NULL,  -- Identifies the tag
    version VARCHAR(20) NOT NULL,  -- Version of the tag
    custom BOOLEAN NOT NULL,  -- Custom field for additional info
    abstract BOOLEAN NOT NULL,  -- Describes the tag
    datatype VARCHAR(20),  -- Data type of the tag
    iord VARCHAR(1),
    crdr VARCHAR(1),
    tlabel VARCHAR(512),
    doc VARCHAR(2048),
    PRIMARY KEY (tag, version)  -- Set a composite primary key on (tag, version)
);


 * postgresql://student@localhost/financial_db
Done.
Done.


[]

**Drop the existing 'TAG' table if it exists, then create a new table 'TAG' with columns for tag identifiers, versioning, and metadata, setting a composite primary key on (tag, version)**


In [7]:
%%sql
DROP TABLE IF EXISTS PRE CASCADE;

CREATE TABLE PRE (
    adsh VARCHAR(20) NOT NULL,  -- Foreign key, references the SUB table
    report NUMERIC(6) NOT NULL,
    line NUMERIC(6) NOT NULL,  -- Line number in the report
    stmt VARCHAR(2),
    inpth BOOLEAN NOT NULL,
    tag VARCHAR(256) NOT NULL,  -- Foreign key, references the TAG table
    version VARCHAR(20) NOT NULL,  -- Foreign key, references the TAG table
    prole VARCHAR(50) NOT NULL,  -- Role of the person creating the report
    plabel VARCHAR(512),  -- Label of the financial data in the report
    negating BOOLEAN NOT NULL,
    PRIMARY KEY (adsh, report, line), -- Set a composite primary key on (adsh, report, line)
    FOREIGN KEY (adsh) REFERENCES SUB(adsh),  -- Foreign key constraint referencing SUB table
    FOREIGN KEY (tag, version) REFERENCES TAG(tag, version)  -- Foreign key constraint referencing TAG table
);


 * postgresql://student@localhost/financial_db
Done.
Done.


[]

**Drop the existing 'PRE' table if it exists, then create a new table 'PRE' with columns for report details, including identifiers, line information, and metadata, setting a composite primary key on (adsh, report, line) and defining foreign key constraints referencing 'SUB' and 'TAG' tables**

In [8]:
%%sql
DROP TABLE IF EXISTS DIM CASCADE;

CREATE TABLE DIM (
    dimh VARCHAR(35) NOT NULL PRIMARY KEY,  -- Primary key, uniquely identifies each dimension
    segment VARCHAR(1024),  -- Segment of the financial data
    segt BOOLEAN NOT NULL
);


 * postgresql://student@localhost/financial_db
Done.
Done.


[]

**Drop the existing 'DIM' table if it exists, then create a new table 'DIM' with columns for dimension identifiers and segment details, setting 'dimh' as the primary key**

In [9]:
%%sql
DROP TABLE IF EXISTS NUM;

CREATE TABLE NUM (
    adsh VARCHAR(20) NOT NULL,  -- Foreign key, references the SUB table
    tag VARCHAR(256) NOT NULL,  -- Foreign key, references the TAG table
    version VARCHAR(20) NOT NULL,  -- Foreign key, references the TAG table
    ddate DATE NOT NULL,  -- Date of the financial data
    qtrs NUMERIC(8) NOT NULL,
    uom VARCHAR(20) NOT NULL,
    dimh VARCHAR(40) NOT NULL,  -- Foreign key, references the DIM table
    iprx NUMERIC(2) NOT NULL,  -- IP-related identifier
    value NUMERIC(16),  -- Numeric value of the financial data
    footnote VARCHAR(512),
    footlen NUMERIC(4) NOT NULL,
    dimn NUMERIC(1) NOT NULL,
    coreg VARCHAR(256),
    durp NUMERIC(7,3) NOT NULL,
    datp NUMERIC(7,3) NOT NULL,
    dcml NUMERIC(5) NOT NULL,
    PRIMARY KEY (adsh, tag, version, ddate, qtrs, uom, dimh, iprx),  -- Composite primary key
    FOREIGN KEY (adsh) REFERENCES SUB(adsh),  -- Foreign key constraint referencing SUB table
    FOREIGN KEY (tag, version) REFERENCES TAG(tag, version),  -- Foreign key constraint referencing TAG table
    FOREIGN KEY (dimh) REFERENCES DIM(dimh)  -- Foreign key constraint referencing DIM table
);


 * postgresql://student@localhost/financial_db
Done.
Done.


[]

**Drop the existing 'NUM' table if it exists, then create a new table 'NUM' for detailed financial data with identifiers, dates, numeric values, and metadata. Define a composite primary key and set foreign key constraints referencing the 'SUB', 'TAG', and 'DIM' tables**

In [1]:
#!wget -O 2024_01_SEC-data.zip https://www.sec.gov/files/dera/data/financial-statement-notes-data-sets/2024_01_notes.zip

In [ ]:
#!unzip -o 2024_01_notes

**Download and extract the data from the SEC website**

Note that this function did not work with the SEC site so our files were downloaded and extracted manually.

In [ ]:
#!csvformat -T dim.tsv > dim.csv
#!csvformat -T num.tsv > num.csv
#!csvformat -T tag.tsv > tag.csv
#!csvformat -T sub.tsv > sub.csv
#!csvformat -T pre.tsv > pre.csv

**Convert the filetype to CSV**

In [10]:
%%sql
COPY SUB FROM '/home/ubuntu/notebooks/homework 2/sub.csv'
CSV
HEADER;

 * postgresql://student@localhost/financial_db
5904 rows affected.


[]

In [11]:
%%sql
COPY TAG FROM '/home/ubuntu/notebooks/homework 2/tag.csv'
CSV
HEADER;

 * postgresql://student@localhost/financial_db
84276 rows affected.


[]

In [12]:
%%sql
COPY PRE FROM '/home/ubuntu/notebooks/homework 2/pre.csv'
CSV
HEADER;

 * postgresql://student@localhost/financial_db
511413 rows affected.


[]

In [13]:
%%sql
COPY DIM FROM '/home/ubuntu/notebooks/homework 2/dim.csv'
CSV
HEADER;

 * postgresql://student@localhost/financial_db
43201 rows affected.


[]

In [14]:
%%sql
COPY NUM FROM '/home/ubuntu/notebooks/homework 2/num.csv'
CSV
HEADER;

 * postgresql://student@localhost/financial_db
598843 rows affected.


[]

**Load data into the proper database tables**

In [1]:
!csvstat --count sub.csv
!csvstat --count tag.csv
#!csvstat --count pre.csv
!csvstat --count dim.csv
#!csvstat --count num.csv

5904
84276
43201


**Verify file row counts equal tables**

Note that the PRE and NUM files require additional processing power as they are so large.

# Part III

**Formulate SQL queries from question prompts**

In [15]:
%%sql
SELECT DISTINCT SUB.adsh submission_id, SUB.name company_name, SUB.filed submission_date,
        SUB.bas1 || ' ' || SUB.bas2 || ' ' || SUB.cityba || ' ' || SUB.zipba || ' ' || SUB.countryba mailing_address
FROM SUB 
WHERE SUB.sic in (6321, 8000, 8082, 8090)

 * postgresql://student@localhost/financial_db
37 rows affected.


submission_id,company_name,submission_date,mailing_address
0000004977-24-000006,AFLAC INC,2024-01-31,None
0000005513-24-000003,UNUM GROUP,2024-01-04,None
0000005513-24-000008,UNUM GROUP,2024-01-30,None
0000885978-24-000002,U S PHYSICAL THERAPY INC /NV,2024-01-09,1300 WEST SAM HOUSTON PARKWAY SUITE 300 HOUSTON 77043 US
0000950170-24-004820,"AVEANNA HEALTHCARE HOLDINGS, INC.",2024-01-17,400 INTERSTATE NORTH PARKWAY STE 1700 ATLANTA 30339 US
0000950170-24-005016,"MARPAI, INC.",2024-01-17,615 CHANNELSIDE DRIVE SUITE 207 TAMPA 33602 US
0000950170-24-005385,"MARPAI, INC.",2024-01-18,615 CHANNELSIDE DRIVE SUITE 207 TAMPA 33602 US
0001104659-24-001643,GREENBROOK TMS INC.,2024-01-05,None
0001104659-24-003303,PRINCIPAL FINANCIAL GROUP INC,2024-01-11,None
0001104659-24-005618,GREENBROOK TMS INC.,2024-01-22,None


**Question 1: Company submission and mailing info for Healthcare SICs**

Note that these SICs were taken directly from the SEC website (https://www.sec.gov/search-filings/standard-industrial-classification-sic-code-list)

In [19]:
%%sql
SELECT case
        WHEN filed BETWEEN '2024-01-01' AND '2024-01-31' THEN 'JAN 2024' 
        WHEN filed BETWEEN '2024-02-01' AND '2024-02-28' THEN 'FEB 2024'
        WHEN filed BETWEEN '2024-02-01' AND '2024-02-28' THEN 'FEB 2024'
        WHEN filed BETWEEN '2024-02-01' AND '2024-02-28' THEN 'FEB 2024'
        ELSE 'INVALID'
        END AS month, count(DISTINCT adsh) total_submissions, count(DISTINCT adsh)/count(DISTINCT name) average_per_company 
FROM SUB
GROUP BY month;

 * postgresql://student@localhost/financial_db
1 rows affected.


month,total_submissions,average_per_company
JAN 2024,5904,1


**Question 2: Compute total submissions per month and average per company**

Note that the average calculation is an assumption as the question prompt was unclear.

In [22]:
%%sql
SELECT NUM.tag tag_id, TAG.tlabel tag_name, count(DISTINCT SUB.name) num_companies
FROM TAG NATURAL JOIN NUM NATURAL JOIN SUB
WHERE SUB.sic in (6021, 6029, 6035, 6036, 6099, 6111, 6141, 6153, 6159, 6162, 
                  6163, 6172, 6189, 6199, 6200, 6211, 6221, 6282, 6311, 6321, 6324, 6331, 6351, 6361, 6399, 6411)
GROUP BY NUM.tag, TAG.tlabel;

 * postgresql://student@localhost/financial_db
2539 rows affected.


tag_id,tag_name,num_companies
AccountingProfit,Accounting profit,1
AccountsAndNotesReceivableNet,"Accounts and Financing Receivable, after Allowance for Credit Loss",1
AccountsAndOtherReceivablesNetCurrent,"Accounts and Other Receivables, Net, Current",1
AccountsNotesAndLoansReceivableNetCurrent,"Accounts and Financing Receivable, after Allowance for Credit Loss, Current",1
AccountsPayableAndAccruedLiabilitiesCurrent,"Accounts Payable and Accrued Liabilities, Current",2
AccountsPayableAndAccruedLiabilitiesCurrentAndNoncurrent,Accounts Payable and Accrued Liabilities,3
AccountsPayableAndOtherAccruedLiabilities,Accounts Payable and Other Accrued Liabilities,2
AccountsPayableCurrent,Accounts payable,6
AccountsReceivableGross,"Accounts Receivable, before Allowance for Credit Loss",1
AccountsReceivableNet,"Accounts Receivable, after Allowance for Credit Loss",1


**Question 3: Finance company tags and counts of use**

Note that we used the same SIC list provided by the SEC (https://www.sec.gov/search-filings/standard-industrial-classification-sic-code-list) and assumed that each company used the same name on all submissions.

In [21]:
%%sql
SELECT sum(SUB.pubfloatUSD) assets, SUB.name company_name, SUB.sic sector, count(DISTINCT NUM.tag) num_tags
FROM SUB NATURAL JOIN NUM
WHERE SUB.pubfloatUSD is not NULL
GROUP BY SUB.name, SUB.sic
ORDER BY assets DESC
LIMIT 5;

 * postgresql://student@localhost/financial_db
5 rows affected.


assets,company_name,sector,num_tags
1883648000000000.00,ALPHABET INC.,7370,334
1054879200000000.00,"TESLA, INC.",3711,434
485800000000000.00,INTEL CORP,3674,407
285661200000000.00,BOEING CO,3721,540
231368500000000.00,ADVANCED MICRO DEVICES INC,3674,332


**Question 4: Display top 5 companies by assets**

Note that Prof Obaidi specifically linked SIC with sector in direct email and there is no table with the industry text available for display.